# 7.1. F1-measure

На [лекции](https://colab.research.google.com/drive/12xpYdqi1S4y68FYHym2rZ3wzHSm8cSLm?usp=sharing) мы подробно обсудили, что доля правильных ответов - не самая лучшая метрика оценки качества классификации. Довольно часто доля правильных ответов даёт смещенную оценку качества и способна ввести в заблуждение. По этой причине мы пользуемся метриками качества, которые называются *точностью* и *полнотой*, а также их комбинацией - *F1-мерой*. Ваша задача - реализовать функции `precision`, `recall` и `f1`. На вход всех этих функий подаются два вектора: `y_true`: вектор правильных ответов и `y_pred`: вектор предсказаний.

В рамках выполнения этого задания можно использовать только модуль `numpy`.

**Замечание:** проверить себя Вы можете, сравнив свои ответы с функциями `sklearn.metrics.precision_score`, `sklearn.metrics.recall_score` и `sklearn.metrics.f1_score`

In [1]:
import numpy as np


def precision(y_true, y_pred):
    TP = y_pred[y_true == y_pred].sum()
    FP = y_pred[y_true != y_pred].sum()
    return TP / (TP + FP)

def recall(y_true, y_pred):
    TP = y_pred[y_true == y_pred].sum()
    FN = - (y_pred[y_true != y_pred] - 1).sum()
    return TP / (TP + FN)

def f1(y_true, y_pred):
    precision_socre = precision(y_true, y_pred)
    recall_score = recall(y_true, y_pred)
    return 2 * (precision_socre * recall_score) / (precision_socre + recall_score)

# 7.2. Площади

Мы предлагаем вам решить задачу бинарной классификации на датасете breast_cancer при помощи 4 различных алгоритмов:
* Decision Tree (1)
* Logistic Regression (2)
* KNN (3)
* SVC (4)

Ваша задача - найти алгоритмы с наилучшими показателями ROC AUC score и PR AUC score. В качестве ответа укажите число ab, где a - лучший с точки зрения ROC AUC score алгоритм, а b - лучший с точки зрения PR AUC score алгоритм.

Например, ответ 43 будет интерпретирован следующим образом: Алгоритм 4 (SVM) оказался лучшим с точки зрения roc_auc_score, а алгоритм 3 (KNN) - с точки зрения pr_auc_score.

В качестве параметров выбирайте параметры по умолчанию. Для KNN выберете k=5. Random State установите равным 42. Разбиение на train и test проведите с параметром test_size=0.3.

Не забудьте провести дополнительную предобработку данных, приведя их к единой шкале при помощи StandardScaler.
Примеры вычисления обеих метрик были предложены на лекции.

В этой задаче Вам требуется сравнить значения метрик для 4 различных алгоритмов. Для этого отредактируйте следующий код так, чтобы он соответствовал сформулированному заданию.

In [2]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve


X, y = load_breast_cancer(return_X_y=True)
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=42)

tree = DecisionTreeClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
knn = KNeighborsClassifier()
svm = SVC(probability=True, random_state=42)

estimators = [tree, lr, knn, svm]

roc_scores = []
pr_scores = []
for estimator in estimators:
    estimator.fit(X_train, y_train)
    y_score = estimator.predict_proba(X_test)[:, 1]
    precision, recall, _ = precision_recall_curve(y_test, y_score)
    
    roc = roc_auc_score(y_test, y_score)        
    pr = auc(recall, precision)
    
    roc_scores.append(roc)
    pr_scores.append(pr)
    
print(np.argmax(roc_scores) + 1, np.argmax(pr_scores) + 1, sep='')

22


# 7.3. Усреднение

Ваша задача в этом задании - написать функции для реализации микро и макро усреднения метрик при многоклассовой классификации. Прочитать про концепции микро- и макроусреднений можно в [лекции 7](https://colab.research.google.com/drive/12xpYdqi1S4y68FYHym2rZ3wzHSm8cSLm#scrollTo=RGxp4ftIBbfG). 

Заполните тела функций:

* `err_table` для составления матрицы ошибок для многоклассового случая
* `micro_precision` для микроусреднения точности
* `macro_precision` для макроусреднения точности
* `micro_recall` для микроусреднения полноты
* `macro_recall` для макроусреднения полноты
* `micro_f1` для микроусреднения f1-меры
* `macro_f1` для макроусреднения f1-меры

В рамках решения этого задания можно пользоваться только фреймворком `numpy`

*Напоминание:*

Микроусреднение подразумевает вычисление совокупной усреднённой матрицы ошибок по всем k бинарным классификациям, каждая из которых отделяет i-й класс от всех остальных. Затем по этой усредненной матрице вычисляется необходимая метрика.

То есть, если мы считаем некоторую функцию $f(a_i, b_i)$, где $a_i, b_i$ - величины, определённые для каждой из k задач классификации, то в случае микроусреднения мы посчитаем значение $\bar{f} = f(\bar{a}, \bar{b})$, где значком $\bar{}$ обозначено среднее значение.

 Макроусреднение же предполагает вычисление k независимых метрик для каждой задачи бинарной классификации, и лишь затем мы усредняем эти значения.
 
 То есть мы должны посчитать набор значений $f_i = f(a_i, b_i)$, а затем усреднить $\bar{f} = \frac{1}{k}Σf_i$
___________________________
**Замечание:**
Мы гарантируем, что в y_true представлены все возможные классы, то есть при составлении таблицы err_table можно ориентироваться на число классов в y_true


In [3]:
import numpy as np


def err_table(y_true: np.array, y_pred: np.array):
    """
    В рамках этой функции Вы должны составить матрицу ошибок предсказаний для многоклассовой классификации
    Строки этой матрицы будут символизировать истинные классы, а столбцы - предсказанные.
    Например, число N в ячейке матрицы ij будет интерпретировано следующим образом: 
      - Данный алгоритм классификации классифицировал N объектов, относящихся к классу i, как объекты класса j
    
    Замечание:
        Функция, выполняющая такую задачу реализована в sklearn.metrics, называется confusion_matrix. 
        В Вашей будущей работе мы рекомендуем пользоваться именно ей. Однако в рамках выполнения этого задания 
        запрещено использование функций из sklearn

    Аргументы:
        - y_true - массив правильных ответов
        - y_pred - массив предсказанных классов
    """
    classes = np.sort(np.unique(y_true))
    confusion_matrix = np.zeros((len(classes), len(classes)), dtype=int)
    for i in classes:
        for j in classes:
            y_pred_j = y_true[y_pred == j]
            y_pred_j_i = y_pred_j[y_pred_j == i]
            confusion_matrix[i, j] = len(y_pred_j_i)
    return confusion_matrix


def micro_precision(err_table: np.array):
    TP = np.diag(err_table)
    FP = err_table.sum(axis=0) - TP
    return TP.mean() / (TP.mean() + FP.mean())

def macro_precision(err_table: np.array):
    TP = np.diag(err_table)
    FP = err_table.sum(axis=0) - TP
    return (TP / (TP + FP)).mean()

def micro_recall(err_table: np.array):
    TP = np.diag(err_table)
    FN = err_table.sum(axis=1) - TP
    return TP.mean() / (TP.mean() + FN.mean())

def macro_recall(err_table: np.array):
    TP = np.diag(err_table)
    FN = err_table.sum(axis=1) - TP
    return (TP / (TP + FN)).mean()

def micro_f1(err_table: np.array):
    TP = np.diag(err_table)
    FP = err_table.sum(axis=0) - TP
    FN = err_table.sum(axis=1) - TP
    precision = TP.mean() / (TP.mean() + FP.mean())
    recall = TP.mean() / (TP.mean() + FN.mean())
    return 2 * precision * recall / (precision + recall)

def macro_f1(err_table: np.array):
    TP = np.diag(err_table)
    FP = err_table.sum(axis=0) - TP
    FN = err_table.sum(axis=1) - TP
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    return (2 * precision * recall / (precision + recall)).mean()

In [4]:
y_true = np.array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 2, 1, 0, 2])
err_ = err_table(y_true, y_pred)
err_

array([[2, 2, 0],
       [3, 1, 0],
       [1, 1, 2]])

In [5]:
print(micro_precision(err_))
print(macro_precision(err_))
print(micro_recall(err_))
print(macro_recall(err_))
print(micro_f1(err_))
print(macro_f1(err_))

0.4166666666666667
0.5277777777777778
0.4166666666666667
0.4166666666666667
0.4166666666666667
0.4388888888888889
